In [91]:
%reset -f

In [92]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import wget
import os

In [93]:
def getDirectoryName(dirName):
    dirName = dirName.replace(',', '')
    dirName = dirName.replace(' ', '_')
    dirName = 'SpringerBooks_'+dirName
    return dirName

In [94]:
df = pd.read_excel('Free+English+textbooks.xlsx')

colsToDrop = ['Product Type', 'Copyright Year', 'Copyright Holder', 'Edition', 'Print ISBN', 'Electronic ISBN', 'Language Collection', 'German Package Name',
              'Series Print ISSN', 'Series Electronic ISSN', 'Series Title', 'Volume Number', 'DOI URL', 'Subject Classification', 'Publisher', 'Imprint', 'eBook Package', 'Language']

df = df.drop(colsToDrop, axis=1)

df['DirectoryName'] = [getDirectoryName(i) for i in df['English Package Name']]

# df.head()

In [85]:
# create a directory for each book category

for dr in set(df['DirectoryName']):
    if os.path.exists(dr):
        'Do Nothing'
    else:
        os.mkdir(dr)

In [86]:
def getSpringerBookPDF(tupleBookDetails):
    booksFolder = tupleBookDetails[4]
    bookURL = tupleBookDetails[3]
    theFileName = tupleBookDetails[0]+' ['+tupleBookDetails[1]+']'
    print('Saving "' + theFileName.upper() + '" to ' + booksFolder)
    thisPage = requests.get(bookURL)
    thisSoup = BeautifulSoup(thisPage.content, "html.parser")
    thisItem = thisSoup.find('a', {'title': 'Download this book in PDF format'})
    theLinkedPDF = 'http://link.springer.com/' + thisItem['href']
    wget.download(theLinkedPDF, '{}/{}.pdf'.format(booksFolder, theFileName))
    print('\n\tDONE\n')

In [87]:
for tp in zip(df['Book Title'], df['Author'], df['English Package Name'], df['OpenURL'], df['DirectoryName']):
    getSpringerBookPDF(tp)